In [1]:
from google.colab import drive
drive.mount('/content/drive')
root_dir = '/content/drive/My Drive'

Mounted at /content/drive


In [2]:
!unzip '/content/drive/MyDrive/NLP_beginner.zip'

Archive:  /content/drive/MyDrive/NLP_beginner.zip
  inflating: NLP_beginner/NLP_Primer_twitter_challenge.ipynb  
  inflating: NLP_beginner/SampleSubmission.csv  
  inflating: NLP_beginner/Test.csv   
  inflating: NLP_beginner/Train.csv  


In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [55]:
df = pd.read_csv('/content/NLP_beginner/Train.csv')
pd.set_option('display.max_columns', None)

In [56]:
df

,tweet_id,safe_text,label,agreement
0,CL1KWCMY,Me &amp; The Big Homie meanboy3000 #MEANBOY #M...,0.0,1.000000
1,E3303EME,I'm 100% thinking of devoting my career to pro...,1.0,1.000000
2,M4IVFSMS,"#whatcausesautism VACCINES, DO NOT VACCINATE Y...",-1.0,1.000000
3,1DR6ROZ4,I mean if they immunize my kid with something ...,-1.0,1.000000
4,J77ENIIE,Thanks to <user> Catch me performing at La Nui...,0.0,1.000000
...,...,...,...,...
9996,IU0TIJDI,Living in a time where the sperm I used to was...,1.0,1.000000
9997,WKKPCJY6,<user> <user> In spite of all measles outbrea...,1.0,0.666667
9998,ST3A265H,Interesting trends in child immunization in Ok...,0.0,1.000000
9999,6Z27IJGD,CDC Says Measles Are At Highest Levels In Deca...,0.0,1.000000


In [57]:
import string
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Remove stop words
    text = ' '.join([word for word in text.split() if word not in ENGLISH_STOP_WORDS])
    return text

In [58]:
df['clean_text'] = df['safe_text'].apply(preprocess_text)

In [67]:
df

,tweet_id,safe_text,label,agreement,clean_text
0,CL1KWCMY,Me &amp; The Big Homie meanboy3000 #MEANBOY #M...,0.0,1.000000,amp big homie meanboy3000 meanboy mb mbs mmr s...
1,E3303EME,I'm 100% thinking of devoting my career to pro...,1.0,1.000000,im 100 thinking devoting career proving autism...
2,M4IVFSMS,"#whatcausesautism VACCINES, DO NOT VACCINATE Y...",-1.0,1.000000,whatcausesautism vaccines vaccinate child
3,1DR6ROZ4,I mean if they immunize my kid with something ...,-1.0,1.000000,mean immunize kid wont secretly kill years lin...
4,J77ENIIE,Thanks to <user> Catch me performing at La Nui...,0.0,1.000000,thanks user catch performing la nuit nyc 1134 ...
...,...,...,...,...,...
9996,IU0TIJDI,Living in a time where the sperm I used to was...,1.0,1.000000,living time sperm used waste jenny mccarthy do...
9997,WKKPCJY6,<user> <user> In spite of all measles outbrea...,1.0,0.666667,user user spite measles outbreaks judge mi thr...
9998,ST3A265H,Interesting trends in child immunization in Ok...,0.0,1.000000,interesting trends child immunization oklahoma...
9999,6Z27IJGD,CDC Says Measles Are At Highest Levels In Deca...,0.0,1.000000,cdc says measles highest levels decades url re...


In [62]:
for i in range(len(df)):
  if df['label'][i] > 0.0 and df['label'][i] < 1.0:
    df['label'][i] = 1
  if df['label'][i] == 'NaN':
    df['label'][i] = 0

<ipython-input-62-a9f88e4466a1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'][i] = 1


In [65]:
df.loc[df['label'].isna(), 'label'] = 0
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10001 entries, 0 to 10000
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   tweet_id    10001 non-null  object 
 1   safe_text   10001 non-null  object 
 2   label       10001 non-null  float64
 3   agreement   9999 non-null   float64
 4   clean_text  10001 non-null  object 
dtypes: float64(2), object(3)
memory usage: 390.8+ KB


In [68]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [69]:
tfidf = TfidfVectorizer(max_features=1000)

X = tfidf.fit_transform(df['clean_text']).toarray()
y = df['label']

In [70]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

y_train.value_counts()

,count
label,
0.0,3921
1.0,3272
-1.0,807


In [71]:
from sklearn.svm import SVC

svc_model = SVC(kernel = 'linear')
svc_model.fit(X_train, y_train)

y_preds = svc_model.predict(X_test)

from sklearn.metrics import confusion_matrix, classification_report
print(confusion_matrix(y_test, y_preds))
print(classification_report(y_test, y_preds))

[[ 49  50 132]
 [ 10 782 196]
 [ 11 163 608]]
              precision    recall  f1-score   support

        -1.0       0.70      0.21      0.33       231
         0.0       0.79      0.79      0.79       988
         1.0       0.65      0.78      0.71       782

    accuracy                           0.72      2001
   macro avg       0.71      0.59      0.61      2001
weighted avg       0.72      0.72      0.70      2001



In [120]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
xgb_model = XGBRegressor(random_state = 42, objective='reg:squarederror')
xgb_model.fit(X_train, y_train)

y_preds = xgb_model.predict(X_test)
print(np.sqrt(mean_squared_error(y_test, y_preds)))


0.6002118442681852


In [110]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_preds)

ValueError: Classification metrics can't handle a mix of multiclass and continuous targets

In [121]:
df_testing = pd.read_csv('/content/NLP_beginner/Test.csv')
df_testing

,tweet_id,safe_text
0,00BHHHP1,<user> <user> ... &amp; 4 a vaccine given 2 he...
1,00UNMD0E,Students starting school without whooping coug...
2,01AXPTJF,"I'm kinda over every ep of <user> being ""rippe..."
3,01HOEQJW,How many innocent children die for lack of vac...
4,01JUKMAO,"CDC eyeing bird flu vaccine for humans, though..."
...,...,...
5172,ZXVVNC5O,jenny mccarthy is on new years rockin eve. wha...
5173,ZYIANVI8,Measles reported in Clark Co. for 1st time sin...
5174,ZYITEHAH,<user> issues alert regarding Measles in TX. K...
5175,ZZ3BMBTG,I can't believe people don't vaccinate their k...


In [122]:
df_testing.loc[df_testing['safe_text'].isna(), 'safe_text'] = 'pula'

In [123]:
df_testing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5177 entries, 0 to 5176
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   tweet_id   5177 non-null   object
 1   safe_text  5177 non-null   object
dtypes: object(2)
memory usage: 81.0+ KB


In [124]:
df_testing['clean_text'] = df_testing['safe_text'].apply(preprocess_text)

In [125]:
X_testing = tfidf.transform(df_testing['clean_text']).toarray()
y_preds = xgb_model.predict(X_testing)

In [126]:
output = pd.DataFrame({'tweet_id': df_testing['tweet_id'], 'label': y_preds})
output.to_csv('submission.csv', index=False)

In [127]:
output['label']

,label
0,-0.265591
1,0.526191
2,0.065838
3,0.290597
4,0.254621
...,...
5172,1.064015
5173,0.475984
5174,0.308442
5175,0.677693


In [128]:
from google.colab import files
files.download('submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>